In [ ]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
#import seaborn as sns

# Data Science
import numpy as np
import pandas as pd

# Multi-dimensional arrays and datasets
import xarray as xr

# Geospatial raster data handling
import rioxarray as rxr

# Geospatial data analysis
#import geopandas as gpd

# Geospatial operations
import rasterio
from rasterio import windows  
from rasterio import features  
from rasterio import warp
from rasterio.warp import transform_bounds 
from rasterio.windows import from_bounds 

# Image Processing
from PIL import Image

# Coordinate transformations
from pyproj import Proj, Transformer, CRS

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

# Planetary Computer Tools
import pystac_client
import planetary_computer as pc
from pystac.extensions.eo import EOExtension as eo

# Others
import os
from tqdm import tqdm





Failed to import TF-Keras. Please note that TF-Keras is not installed by default when you install TensorFlow Probability. This is so that JAX-only users do not have to install TensorFlow or TF-Keras. To use TensorFlow Probability with TensorFlow, please install the tf-keras or tf-keras-nightly package.
This can be be done through installing the tensorflow-probability[tf] extra.




ModuleNotFoundError: No module named 'tf_keras'

In [60]:
def combine_two_datasets(dataset1,dataset2):
    '''
    Returns a  vertically concatenated dataset.
    Attributes:
    dataset1 - Dataset 1 to be combined 
    dataset2 - Dataset 2 to be combined
    '''
    
    data = pd.concat([dataset1,dataset2], axis=1)
    return data

In [ ]:
# Extracts satellite band values from a GeoTIFF based on coordinates from a csv file and returns them in a DataFrame.

def map_satellite_data(tiff_path, csv_path):
    
    # Load the GeoTIFF data
    data = rxr.open_rasterio(tiff_path)
    tiff_crs = data.rio.crs

    # Read the Excel file using pandas
    df = pd.read_csv(csv_path)
    latitudes = df['Latitude'].values
    longitudes = df['Longitude'].values

    # 3. Convert lat/long to the GeoTIFF's CRS
    # Create a Proj object for EPSG:4326 (WGS84 - lat/long) and the GeoTIFF's CRS
    proj_wgs84 = Proj(init='epsg:4326')  # EPSG:4326 is the common lat/long CRS
    proj_tiff = Proj(tiff_crs)
    
    # Create a transformer object
    transformer = Transformer.from_proj(proj_wgs84, proj_tiff)

    vals = [[] for _ in range(n_inputs)]


# Iterate over the latitudes and longitudes, and extract the corresponding band values
    for lat, lon in tqdm(zip(latitudes, longitudes), total=len(latitudes), desc="Mapping values"):
    # Assuming the correct dimensions are 'y' and 'x' (replace these with actual names from data.coords)
    
        for i in range(n_inputs):
            tmp = data.sel(x=lon, y=lat, band=i, method='nearest').values
            vals[i].append(tmp)

    # Create a DataFrame with the band values
    # Create a DataFrame to store the band values
    df = pd.DataFrame()
    for i in range(n_inputs):
        df[names[i]] = vals[i]
    
    return df


In [88]:
# Constants
n_inputs = 28
names = ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B11", "B12", "ndwi", "ndvi", "ndbi", 
        "B01 (pooled)", "B02 (pooled)", "B03 (pooled)", "B04 (pooled)", "B05 (pooled)", "B06 (pooled)", "B07 (pooled)", "B08 (pooled)", "B8A (pooled)", "B11 (pooled)", "B12 (pooled)", "ndwi (pooled)", "ndvi (pooled)", "ndbi (pooled)"]
UhiData_FileName = "Data/Training_data_uhi_index.csv"
satellite_data =  "Data/S2_median_fullBands_indeces.tiff"

# Open the GeoTIFF file
tiff_path = "Data/S2_median_fullBands_indeces.tiff"

# Read the bands from the GeoTIFF file
with rasterio.open(tiff_path) as src1:
    cols = [src1.read(i) for i in range(1, n_inputs+1)]


In [ ]:
# Would be faster in a df
feature_data = map_satellite_data(satellite_data, UhiData_FileName)

Mapping values: 100%|██████████| 11229/11229 [08:38<00:00, 21.64it/s]


In [89]:
# Redo columns
feature_data["ndwi"] = (feature_data["B03"] - feature_data["B08"])/(feature_data["B03"] + feature_data["B08"]) #Normalized Difference Water Index
feature_data["ndbi"] = (feature_data["B11"] - feature_data["B08"])/(feature_data["B11"] + feature_data["B08"]) # Normalized Difference Buildup Index
feature_data["ndvi"] = (feature_data["B08"] - feature_data["B04"])/(feature_data["B08"] + feature_data["B04"]) #Normalized Difference Vegetation Index
feature_data["ndwi (pooled)"] = (feature_data["B03 (pooled)"] - feature_data["B08 (pooled)"])/(feature_data["B03 (pooled)"] + feature_data["B08 (pooled)"]) #Normalized Difference Water Index
feature_data["ndbi (pooled)"] = (feature_data["B11 (pooled)"] - feature_data["B08 (pooled)"])/(feature_data["B11 (pooled)"] + feature_data["B08 (pooled)"]) # Normalized Difference Buildup Index
feature_data["ndvi (pooled)"] = (feature_data["B08 (pooled)"] - feature_data["B04 (pooled)"])/(feature_data["B08 (pooled)"] + feature_data["B04 (pooled)"]) #Normalized Difference Vegetation Index

feature_data['ndvi'] = feature_data['ndvi'].replace([np.inf, -np.inf], np.nan) 
feature_data['ndwi'] = feature_data['ndwi'].replace([np.inf, -np.inf], np.nan) 
feature_data['ndbi'] = feature_data['ndbi'].replace([np.inf, -np.inf], np.nan)
feature_data['ndvi (pooled)'] = feature_data['ndvi (pooled)'].replace([np.inf, -np.inf], np.nan) 
feature_data['ndwi (pooled)'] = feature_data['ndwi (pooled)'].replace([np.inf, -np.inf], np.nan) 
feature_data['ndbi (pooled)'] = feature_data['ndbi (pooled)'].replace([np.inf, -np.inf], np.nan) 

In [90]:
# Combining ground data and feature data into a single dataset.
ground_df = pd.read_csv(UhiData_FileName)

uhi_data = combine_two_datasets(ground_df,feature_data)

In [91]:
# Remove duplicate rows from the DataFrame based on specified columns and keep the first occurrence
columns_to_check = ['B01','B04','B06','B08','ndvi']
for col in columns_to_check:
    # Check if the value is a numpy array and has more than one dimension
    uhi_data[col] = uhi_data[col].apply(lambda x: tuple(x) if isinstance(x, np.ndarray) and x.ndim > 0 else x)

# Now remove duplicates
uhi_data = uhi_data.drop_duplicates(subset=columns_to_check, keep='first')
uhi_data.head()

,Longitude,Latitude,datetime,UHI Index,B01,B02,B03,B04,B05,B06,...,B05 (pooled),B06 (pooled),B07 (pooled),B08 (pooled),B8A (pooled),B11 (pooled),B12 (pooled),ndwi (pooled),ndvi (pooled),ndbi (pooled)
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,841.5,841.5,1053.0,1155.0,1206.0,1481.5,...,1220.8703703703707,1428.3641975308642,1753.5802469135792,1869.7654320987654,1814.5370370370372,1927.7037037037044,1841.3209876543203,-0.308904,0.247049,0.015257
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,841.5,841.5,646.0,823.0,777.0,1130.5,...,1134.7777777777778,1396.148148148148,1812.469135802468,1948.9567901234568,1876.382716049383,2021.0740740740753,1793.4320987654314,-0.354534,0.292568,0.018165
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,841.5,841.5,625.0,766.0,741.5,1130.5,...,1165.3765432098771,1413.3148148148139,1846.5679012345677,1988.7777777777778,1953.9691358024688,2069.808641975309,1796.4876543209875,-0.344815,0.284392,0.019965
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,841.5,841.5,659.5,763.0,708.5,1077.5,...,1197.7777777777776,1426.7469135802453,1871.851851851852,2020.4691358024688,2023.9629629629626,2102.641975308642,1816.3395061728395,-0.331936,0.272227,0.019930
6,-73.909312,40.812710,24-07-2021 15:53,1.015143,841.5,841.5,551.5,768.5,659.0,1077.5,...,1082.1975308641972,1356.6790123456778,1823.4012345679012,1985.0925925925926,1991.2037037037037,2068.808641975308,1777.5370370370372,-0.361310,0.300358,0.020651


In [93]:
# Load data, normalize distributions
model1_data = uhi_data[names + ['UHI Index']]
md1_data_std = model1_data.apply(lambda x: (x - x.mean()) / x.std(), axis=0) 
md1_data_std =md1_data_std.apply(lambda x: pd.to_numeric(x, errors = 'coerce'), axis=0) 

# Convert object columns to numeric
#for col in ['B02', 'B03', 'B05', 'B11', 'B02 (pooled)', 'B03 (pooled)', 'B05 (pooled)', 'B11 (pooled)']:
for col in names:
    na_count = md1_data_std[col].isna().sum()
    #print(f"Number of NaN values in {col} after conversion: {na_count}")
    
#print(md1_data_std.dtypes)

In [94]:
# Divide data into training and testing
X = md1_data_std.drop(columns=['UHI Index']).values
y = md1_data_std['UHI Index'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=123)
print(X_train.shape) 
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5848, 28)
(5848,)
(2507, 28)
(2507,)


In [111]:
# Model
model1 = Sequential([
    Dense(32, activation = 'relu', input_shape=(28,) ),
    Dropout(0.1),
    Dense(32, activation = 'relu'),
    Dropout(0.1),
    Dense(1, activation = 'linear'),
])
model1.compile(optimizer=Adam(learning_rate=0.001) , loss = "MSE" )
model1.fit(X_train, y_train, epochs=50) 

Epoch 1/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.2105
Epoch 2/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9056
Epoch 3/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8627
Epoch 4/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8621
Epoch 5/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8619
Epoch 6/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8256
Epoch 7/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8196
Epoch 8/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8270
Epoch 9/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8114
Epoch 10/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8080
Epoch 11/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.8027
Epoch 12/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8372
Epoch 13/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.7874
Epoch 14/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8095
Epoch 15/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - lo

In [112]:
print(f"MSE: {mean_squared_error(y_test , model1.predict(X_test))}")
print(f"r2: {r2_score(y_test, model1.predict(X_test))}")

79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
MSE: 0.7739499633973802
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
r2: 0.22736898960834884


In [104]:
y_lazy_pred = np.mean(y_test)*np.ones(len(y_test))
print(f'Lazy MSE: {mean_squared_error(y_test , y_lazy_pred)}')
print(f"Lazy r2: {r2_score(y_test, y_lazy_pred)}")

Lazy MSE: 1.0017070930211052
Lazy r2: 0.0


In [29]:
md2_data_std = md1_data_std[['B01', 'B05' , 'B08', 'B11','ndwi' , 'ndvi', 'ndbi' , 'UHI Index']]

X2 = md2_data_std.drop(columns=['UHI Index']).values
y2 = md2_data_std['UHI Index'].values


X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3,random_state=123)
print(X_train.shape) 
print(y_test.shape)

(5847, 10)
(2507,)


In [ ]:
#model using only the features used in the sample notebook 

model2 = Sequential([
    #Dense(512 , input_shape=(7,), activation='relu'),
    #Dense(128, activation = 'relu'),
    Dense(30, activation = 'linear',
        kernel_regularizer=regularizers.l2(0.01)),
    Dense(30, activation = 'relu'),
    Dropout(0.2), 
    Dense(30, activation = 'linear',
        kernel_regularizer=regularizers.l2(0.01)),
    Dense(30, activation = 'relu'),
    Dropout(0.2),
    Dense(30, activation = 'linear',
        kernel_regularizer=regularizers.l2(0.01)),
    Dense(30, activation = 'relu'),
    Dropout(0.2),
    Dense(30, activation = 'linear',
        kernel_regularizer=regularizers.l2(0.01)),
    Dense(30, activation = 'relu'),
    Dropout(0.2),
    Dense(30, activation = 'linear',
        kernel_regularizer=regularizers.l2(0.01)),
    Dense(30, activation = 'relu'),
    Dropout(0.2),
    Dense(30, activation = 'linear',
        kernel_regularizer=regularizers.l2(0.01)),
    Dense(30, activation = 'relu'),
    Dropout(0.2),
    Dense(30, activation = 'linear'),
    Dense(30, activation = 'relu'),
    #Dense(64 , activation = 'relu'),
    #Dense(32, activation = 'relu'),
    #Dense(16, activation = 'relu' ),
    Dense(1, activation = 'linear' ) #Output continous variable UHI index 
])

model2.compile(optimizer= 'adam' , loss = "MSE" )

model2.fit(X2_train, y2_train, epochs=50 ,batch_size=10, validation_split=0.2 ) 

#model_lassoreg.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)



Epoch 1/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 1.9606 - val_loss: 1.1133
Epoch 2/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 1.0727 - val_loss: 0.9809
Epoch 3/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.9482 - val_loss: 0.9687
Epoch 4/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.9465 - val_loss: 0.9452
Epoch 5/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.9378 - val_loss: 0.9232
Epoch 6/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.9059 - val_loss: 0.9438
Epoch 7/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.9422 - val_loss: 0.9455
Epoch 8/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.9184 - val_loss: 0.9344
Epoch 9/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.9005 - val_loss: 0.9348
Epoch 10/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.9500 - val_loss: 0.9273
Epoch 11/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.9423 - val_loss: 1.0096
Epoch 12/50
468/468 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/ste

In [26]:
print(r2_score(y_test, model2.predict(X2_test)))

53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 627us/step
0.11947834753868347


In [ ]:
#Model with a regularizer to increas epocs and avoid overfitting

from tensorflow.keras import regularizers



model_lassoreg = Sequential([
    Dense(512 , input_shape=(10,), activation='relu'),
    Dropout(0.2),
    Dense(128, activation = 'linear' ,
        kernel_regularizer=regularizers.l2(0.01)),
    #Dense(30, activation = 'relu'),
    Dense(64 , activation = 'relu' ,
         kernel_regularizer=regularizers.l2(0.1)),
    Dense(32, activation = 'linear',
         kernel_regularizer=regularizers.l2(0.01)),
    Dense(16, activation = 'relu' ,
         kernel_regularizer=regularizers.l2(0.1)),
    Dense(1, activation = 'linear' ) #Output continous variable UHI index 
])


model_lassoreg.compile(optimizer= 'adam' , loss = "MSE" )

model_lassoreg.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 55.2669 - val_loss: 4.3807
Epoch 2/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.0770 - val_loss: 1.1767
Epoch 3/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1342 - val_loss: 1.1101
Epoch 4/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0740 - val_loss: 1.1024
Epoch 5/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1046 - val_loss: 1.1014
Epoch 6/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0829 - val_loss: 1.0996
Epoch 7/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0937 - val_loss: 1.1005
Epoch 8/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0682 - val_loss: 1.0999
Epoch 9/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0968 - val_loss: 1.0991
Epoch 10/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1185 - val_loss: 1.1013
Epoch 11/100
168/168 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1217 - val_loss: 1.0993
Epoch 12/100
168/168 ━━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [27]:
print(r2_score(y_test, model_lassoreg.predict(X_test)))

53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 642us/step
0.1269695466287537


In [22]:
#attempt at helping the network find nonlinear relations
import itertools

md3_data = md1_data_std.copy()
feature_list = md3_data.drop(columns=['UHI Index']).columns.tolist()

column_combinations = itertools.combinations(feature_list, 2)
print(columns_to_check)
for col1, col2 in column_combinations:
    new_col_name = f'{col1}_{col2}'  # Name for the new column
    md3_data[new_col_name ] = md3_data[col1] * md3_data[col2]  # Multiply the column



len(feature_list)
md3_data.columns.to_list()
#md3_data.shape

['B01', 'B04', 'B06', 'B08', 'ndvi']


['B01',
 'B02',
 'B03',
 'B04',
 'B05',
 'B08',
 'B11',
 'ndwi',
 'ndvi',
 'ndbi',
 'UHI Index',
 'B01_B02',
 'B01_B03',
 'B01_B04',
 'B01_B05',
 'B01_B08',
 'B01_B11',
 'B01_ndwi',
 'B01_ndvi',
 'B01_ndbi',
 'B02_B03',
 'B02_B04',
 'B02_B05',
 'B02_B08',
 'B02_B11',
 'B02_ndwi',
 'B02_ndvi',
 'B02_ndbi',
 'B03_B04',
 'B03_B05',
 'B03_B08',
 'B03_B11',
 'B03_ndwi',
 'B03_ndvi',
 'B03_ndbi',
 'B04_B05',
 'B04_B08',
 'B04_B11',
 'B04_ndwi',
 'B04_ndvi',
 'B04_ndbi',
 'B05_B08',
 'B05_B11',
 'B05_ndwi',
 'B05_ndvi',
 'B05_ndbi',
 'B08_B11',
 'B08_ndwi',
 'B08_ndvi',
 'B08_ndbi',
 'B11_ndwi',
 'B11_ndvi',
 'B11_ndbi',
 'ndwi_ndvi',
 'ndwi_ndbi',
 'ndvi_ndbi']

In [23]:
X3 = md3_data.drop(columns=['UHI Index']).values
y3 = md3_data['UHI Index'].values


X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.2,random_state=125)
print(X3_train.shape) 
print(y3_test.shape)


(6683, 55)
(1671,)


In [24]:
from tensorflow.keras import regularizers


quartic_md_reg = Sequential([
    Dense(2048 , input_shape=(55,), activation='linear'),
    Dropout(0.2),
    Dense(1024 , activation='relu'),
    Dropout(0.2),
    Dense(128, activation = 'linear' ,
        kernel_regularizer=regularizers.l2(0.5)),
    Dense(64 , activation = 'relu' ,
         kernel_regularizer=regularizers.l2(0.5)),
    Dense(32, activation = 'linear',
         kernel_regularizer=regularizers.l2(0.5)),
    Dense(16, activation = 'relu' ,
         kernel_regularizer=regularizers.l2(0.5)),
    Dense(1, activation = 'linear' ) #Output continous variable UHI index 
])

#this architecture achieves the same result as the original DNN. 
'''quartic_md_reg = Sequential([
    Dense(2048 , input_shape=(55,), activation='linear'),
    Dropout(0.2),
    Dense(1024 , activation='relu'),
    Dropout(0.2),
    Dense(128, activation = 'linear' ,
        kernel_regularizer=regularizers.l2(0.5)),
    Dense(64 , activation = 'relu' ,
         kernel_regularizer=regularizers.l2(0.5)),
    Dense(32, activation = 'linear',
         kernel_regularizer=regularizers.l2(0.5)),
    Dense(16, activation = 'relu' ,
         kernel_regularizer=regularizers.l2(0.5)),
    Dense(1, activation = 'linear' ) #Output continous variable UHI index 
])''' 

quartic_md_reg.compile(optimizer= 'adam' , loss = "MSE" )

quartic_md_reg.fit(X3_train, y3_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
535/535 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 47.9798 - val_loss: 1.2515
Epoch 2/50
535/535 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 1.0417 - val_loss: 1.0741
Epoch 3/50
535/535 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.9677 - val_loss: 1.0730
Epoch 4/50
535/535 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 1.0026 - val_loss: 1.0732
Epoch 5/50
535/535 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 1.0141 - val_loss: 1.0730
Epoch 6/50
535/535 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 1.0130 - val_loss: 1.0731
Epoch 7/50
535/535 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 1.0222 - val_loss: 1.0733
Epoch 8/50
535/535 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.9925 - val_loss: 1.0731
Epoch 9/50
535/535 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - loss: 1.0222 - val_loss: 1.0735
Epoch 10/50
535/535 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 1.0239 - val_loss: 1.0730
Epoch 11/50
535/535 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 0.9936 - val_loss: 1.0730
Epoch 12/50
535/535 ━━━━━━━━━━━━━━━━━━━━

In [28]:
print(r2_score(y3_test, quartic_md_reg.predict(X3_test)))

53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
-0.0002088499122305798
